This is our grid search document! Note that there are some keyboard interrupts in our output, because we ran everything again just to make sure it works prior to our final submission! Because the full search takes a long time, we interrupted the kernel. The code works!

In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.errors_impl import InvalidArgumentError
import matplotlib.pyplot as plt

In [9]:
# Preprocessing + Training

# Load the dataset
df = pd.read_csv("../backend/Datasets/COALINDIA.csv")

# Feature selection and preprocessing
dropped_features = ['Date', 'Symbol', 'Series', 
                    'Trades', 'Turnover', 'Deliverable Volume', 
                    '%Deliverble', 'Last', 'VWAP', 'Prev Close']
df.drop(dropped_features, axis=1, inplace=True)

# Define features (X) and target (Y)
X = df.drop('Close', axis=1)
Y = df['Close']

# Scale the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X.values)
Y = scaler_Y.fit_transform(Y.values.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=False)

print(f"X_train - {X_train}")
print("*" * 50)
print(f"X_test - {X_test}")
print("*" * 50)

print(f"Y_train - {Y_train}")
print("*" * 50)

print(f"Y_test - {Y_test}")


X_train - [[0.53912913 0.69460631 0.5541304  1.        ]
 [0.69474787 0.72926939 0.71293327 0.06651257]
 [0.72108335 0.72747647 0.6717056  0.09779414]
 ...
 [0.40011971 0.40475123 0.4025042  0.02255257]
 [0.40026934 0.39489018 0.38616583 0.04220904]
 [0.36465659 0.36261766 0.36784242 0.02943177]]
**************************************************
X_test - [[0.36555439 0.37950097 0.37196519 0.01550283]
 [0.37827323 0.38936202 0.38097419 0.01553931]
 [0.37572946 0.37546691 0.37883646 0.00991563]
 ...
 [0.05132426 0.05079934 0.05481753 0.01289286]
 [0.05656142 0.05259226 0.05527561 0.01735301]
 [0.04952865 0.06618856 0.05329058 0.0570686 ]]
**************************************************
Y_train - [[0.69701067]
 [0.71894247]
 [0.66155926]
 ...
 [0.39762656]
 [0.37990086]
 [0.36472886]]
**************************************************
Y_test - [[3.80501728e-01]
 [3.78248460e-01]
 [3.72239748e-01]
 [3.77196936e-01]
 [3.80501728e-01]
 [3.78098242e-01]
 [3.78098242e-01]
 [3.71188223e-01]

In [ ]:
def create_model_A(activation='relu', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, dropout=0.15):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(hidden, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [ ]:
ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for dropout in [0.05, 0.1, 0.15]:
                        try:
                            mse, r2 = create_model_A(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, dropout=dropout)
                            ret.append([activation, epochs, batch_size, op_learning_rate, hidden, dropout, mse, r2])
                        except InvalidArgumentError as e:
                            print("Model error with params")

    

In [ ]:
print("Best MSE for ANN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for ANN:", sorted(ret, key=lambda x:-x[-1])[0])

In [6]:
def create_model_P(activation=None, epochs=100, batch_size=32, op_learning_rate=1e-5):
    model = Sequential()
    model.add(Dense(1, input_dim=X_train.shape[1], activation=activation))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    # Calculate MSE
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error (Perceptron): {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2


In [10]:
ret = []
for activation in [None, 'relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                try:
                    mse, r2 = create_model_P(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate)
                    ret.append([activation, epochs, batch_size, op_learning_rate, mse, r2])
                except InvalidArgumentError as e:
                    print("Model error with params")


17/17 [==============================] - 0s 316us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 1392.6992577453946
R^2: 0.28688818
----------------------------------------------------------------
17/17 [==============================] - 0s 298us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 5276.005876701099
R^2: -1.7015038
----------------------------------------------------------------
17/17 [==============================] - 0s 275us/step


Activation: None
Epoch: 50
Batch Size: 24
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 10.61555730477912
R^2: 0.9945645
----------------------------------------------------------------
17/17 [==============================] - 0s 327us/step


Activation: None
Epoch: 50
Batch Size: 32
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 526.2100493607261
R^2: 0.7305616
----------------------------------------------------------------
17/17 [==============================] - 0s 293us/step


...

Activation: sigmoid
Epoch: 75
Batch Size: 40
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 14022.4445768852
R^2: -6.179993
----------------------------------------------------------------
17/17 [==============================] - 0s 314us/step


Activation: sigmoid
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.0001
Mean Squared Error (Perceptron): 19261.420051346664
R^2: -8.862537
----------------------------------------------------------------
17/17 [==============================] - 0s 325us/step


Activation: sigmoid
Epoch: 75
Batch Size: 40
Optimizer learning rate: 0.001
Mean Squared Error (Perceptron): 1378.273430309472
R^2: 0.29427463
----------------------------------------------------------------
17/17 [==============================] - 0s 336us/step


Activation: sigmoid
Epoch: 100
Batch Size: 24
Optimizer learning rate: 1e-05
Mean Squared Error (Perceptron): 15430.521115378418
R^2: -6.9009786
----------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
print("Best MSE for Perceptron:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for Perceptron:", sorted(ret, key=lambda x:-x[-1])[0])

In [ ]:
def create_model_D(activation='tanh', epochs=100, batch_size=32, op_learning_rate=1e-5, hidden=128, layers=11, dropout=0.1):
    # Build a neural network model with 2 hidden layers
    # You can experiment with different architectures, including the number of layers and neurons.
    # Building DNN model with 11 hidden layers
    model = Sequential()

    # Input layer
    # Input layer will have same number of neurons as number of feature variables
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))

    # Hidden layers
    # Play around with number of neurons in each hidden layer.
    # Too many neurons leads to overcomplexity, not enough means too simple
    # Tanh activation function here is used b/c it is recommended to use
    # when there are more hidden layers.
    for _ in range(layers-1):
        model.add(Dense(hidden, activation=activation))
        model.add(Dropout(dropout)) # This helps with preventing overfitting

    # Output layer
    # Output layer will have 1 neuron b/c there's only 1 target variable
    model.add(Dense(1))

    # Compile the model
    optimizer = Adam(learning_rate=op_learning_rate)  # Experiment with learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test), verbose=0)

    # Make predictions
    Y_pred = model.predict(X_test)

    # Inverse transform the scaled values
    Y_test_original = scaler_Y.inverse_transform(Y_test)
    Y_pred_original = scaler_Y.inverse_transform(Y_pred)

    print(f"Activation: {activation}\nEpoch: {epochs}\nBatch Size: {batch_size}\nOptimizer learning rate: {op_learning_rate}")
    print(f"Hidden: {hidden}\nLayers: {layers}\nDropout: {dropout}")
    # Calculate MSE and R2
    mse = mean_squared_error(Y_test_original, Y_pred_original)
    print(f"Mean Squared Error: {mse}")
    metric = tf.keras.metrics.R2Score()
    metric.update_state(Y_test_original, Y_pred_original)
    r2 = metric.result().numpy()
    print("R^2:", metric.result().numpy())
    print("-"*64)
    return mse, r2

In [ ]:
m, r = create_model_D()

In [ ]:
# GRID SEARCH

ret = []
for activation in ['relu', 'sigmoid', 'tanh']:
    for epochs in [50, 75, 100, 150]:
        for batch_size in [24, 32, 40]:
            for op_learning_rate in [1e-5, 1e-4, 1e-3]:
                for hidden in [32, 64, 128]:
                    for layers in [6, 9, 11]:
                        for dropout in [0.05, 0.1, 0.15]:
                            try:
                                mse, r2 = create_model_D(activation=activation, epochs=epochs, batch_size=batch_size, op_learning_rate=op_learning_rate, hidden=hidden, layers=layers, dropout=dropout)
                                ret.append([activation, epochs, batch_size, op_learning_rate, hidden, layers, dropout, mse, r2])
                            except:
                                print("Model error with params")



In [ ]:
print("Best MSE for DNN:", sorted(ret, key=lambda x:x[-2])[0])
print("Best R^2 for DNN:", sorted(ret, key=lambda x:-x[-1])[0])